# Analyze results
Aug 28, 2020

In [31]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [32]:
from ipywidgets import *

In [33]:
%matplotlib widget

In [34]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [35]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/')
import post_analysis_pandas as post


In [36]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


In [37]:
img_size=128

### Read validation data

In [38]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=2000

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy


In [39]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

## Read data

In [40]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [41]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20210*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210415_140527_gclip1.0', '20210412_70407_ddp_code_…

In [42]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210416_220730_2dcgan_nofm


## Plot Losses

In [43]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [44]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
11631,11631.0,59.0,0.099703,0.390075,0.489778,5.435154,87.363831,81.928680,-2.483587,81.463715,-2.187976,NaN,NaN,3.589110,-1.201919,-5.428631,0.251728
11632,11632.0,59.0,0.647418,0.118848,0.766265,1.582811,87.084000,85.501190,-2.494313,82.411079,-2.234637,NaN,NaN,0.529394,-4.515712,-1.171993,0.252762
11633,11633.0,59.0,0.077447,0.833238,0.910685,7.078547,90.061272,82.982727,-2.256751,82.383087,-2.312713,NaN,NaN,4.161954,-0.299231,-7.077035,0.254005
11634,11634.0,59.0,0.614482,0.093163,0.707645,2.716074,83.773308,81.057236,-2.424295,82.171120,-2.319833,NaN,NaN,0.647366,-5.454477,-2.569611,0.253037
11635,11635.0,59.0,0.099832,0.332316,0.432148,4.217628,86.736534,82.518906,-2.549126,83.025238,-2.595705,NaN,NaN,3.854841,-1.396589,-4.188724,0.256073
11636,11636.0,59.0,0.158834,0.144423,0.303257,4.705796,84.646988,79.941193,-2.035212,82.822952,-2.661784,NaN,NaN,2.873024,-3.442475,-4.687380,0.250726
11637,11637.0,59.0,0.219674,0.147007,0.366681,3.703866,88.410255,84.706390,-2.546499,82.146584,-2.378964,NaN,NaN,2.438674,-3.321661,-3.648366,0.252863
11638,11638.0,59.0,0.135662,0.156461,0.292123,3.372172,83.265388,79.893219,-2.136872,83.121155,-2.711263,NaN,NaN,2.782053,-3.115778,-3.306294,0.267755
11639,11639.0,59.0,0.123835,0.282355,0.406190,4.606705,86.115982,81.509277,-2.646215,80.580116,-2.327295,NaN,NaN,3.287144,-1.912690,-4.591571,0.258575
11640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.910919,-2.831956,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [46]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-2.187987947463989


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
9951,9951.0,51.0,0.603922,0.083256,0.687177,3.076015,86.470650,83.394638,-2.941351,82.492615,-2.997058,NaN,NaN,0.739911,-6.739561,-2.962016,0.269854
10336,10336.0,53.0,0.380071,0.098812,0.478883,2.129522,82.736458,80.606934,-2.467524,82.670425,-2.979875,NaN,NaN,1.415996,-3.230532,-1.884717,0.261145
11344,11344.0,58.0,0.121743,0.175024,0.296767,4.026037,86.893089,82.867050,-2.617358,83.135422,-2.960986,NaN,NaN,3.152675,-2.821833,-3.999190,0.261473
9952,9952.0,51.0,0.113049,0.280195,0.393244,3.319097,86.951187,83.632088,-2.509281,81.015465,-2.950995,NaN,NaN,4.877656,-1.957356,-3.258833,0.270775
11343,11343.0,58.0,0.096438,0.139296,0.235735,3.791394,87.386688,83.595291,-2.532921,81.787766,-2.934534,NaN,NaN,3.816554,-2.743623,-3.754469,0.265689
10233,10233.0,52.0,0.134452,0.287914,0.422366,4.227657,85.318512,81.090851,-2.765439,81.863029,-2.933542,NaN,NaN,3.213856,-1.706245,-4.205202,0.252728
10335,10335.0,53.0,0.115327,0.309406,0.424733,4.651691,86.659439,82.007751,-2.797702,81.077164,-2.918460,NaN,NaN,3.669508,-1.587494,-4.632107,0.260448
9953,9953.0,51.0,0.106918,0.248682,0.355600,5.357001,87.761269,82.404266,-2.799943,82.414124,-2.912656,NaN,NaN,4.410795,-2.006106,-5.348906,0.269592
9970,9970.0,51.0,0.228702,0.150186,0.378888,3.348208,82.851913,79.503708,-2.764338,80.166443,-2.910434,NaN,NaN,2.524941,-3.273624,-3.289565,0.271766
10234,10234.0,52.0,0.246839,0.140555,0.387395,2.885853,84.398186,81.512329,-2.584117,81.073090,-2.908159,NaN,NaN,1.911441,-3.478012,-2.770654,0.252894


In [47]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

In [48]:
# # bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

# bins=f_transform(bins)   ### scale to (-1,1)
# # ### Extract validation data
# sigma_list=[0.5,0.65,0.8,1.1]
# bkgnd_dict={}
# num_bkgnd=2000

# for label in labels_lst:
#     fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
#     print(fname)
#     samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
#     dict_val=post.f_compute_hist_spect(samples,bins)
#     bkgnd_dict[str(sigma_lst[label])]=dict_val
#     del samples

## Read stored chi-squares for images

In [58]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [59]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [60]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list



### Slice best steps

In [61]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1c']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [62]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [54]:
df_sliced=w.result
# df_sliced

In [63]:
best_step=[]
cut_off=1.0
best_step.append(f_slice_merged_df(df_merged,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=cut_off,sort_col='sum_chi_1c',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[ 5390  9440  9740 10170 10180 10230 10590 11140 11520 11640]


array([ 5390,  9440,  9740, 10170, 10180, 10230, 10590, 11140, 11520,
       11640])

In [64]:
# best_step=[6176]
# best_step= [32300, 35810, 36020, 37030, 38640, 42480, 43850]

# best_step=np.arange(40130,40135).astype(int)

In [65]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 61)
[(27, 5390), (48, 9440), (50, 9740), (52, 10170), (52, 10180), (52, 10230), (54, 10590), (57, 11140), (59, 11520), (59, 11640)]


In [66]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1c
538,27-5390,0.360076,40.798587,0.000115
943,48-9440,0.766020,2.122476,0.000115
973,50-9740,0.101844,6.167125,0.000115
1016,52-10170,0.161930,2.214600,0.000115
1017,52-10180,1.312565,2.557911,0.000115
1022,52-10230,0.119711,7.350897,0.000115
1058,54-10590,0.083734,4.454220,0.000115
1113,57-11140,0.119863,5.224838,0.000115
1151,59-11520,0.227218,2.081810,0.000115
1163,59-11640,0.148171,2.810133,0.000115


### Interactive plot

In [67]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']

                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)
                plt.legend()

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:]
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [68]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type)>

In [31]:
# ### Check deterministic
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# epoch,step=0,230
# f1='20210113_185116_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210113_092234_cgan_predict_0.65_m2/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# epoch,step=3,20
# f1='20210114_191648_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)
# f2='20210114_193009_nb_test/images/gen_img_label-0.5_epoch-{0}_step-{1}.npy'.format(epoch,step)

# a1=np.load(main_dir+f1)
# a2=np.load(main_dir+f2)
# # print(a1.shape,a2.shape)

# print(np.mean(a1),np.mean(a2))
# print(np.max(a1),np.max(a2))

### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [129]:
# fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
fldr=result_dir
print(fldr)
flist=glob.glob(fldr+'/models/checkpoint_*.tar')
len(flist)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210205_063709_cgan_predict_1.1_m2


1767

In [1]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print("Found")
# #             print(step)
# #             os.remove(i)
#             pass
#         else: 
#             print(step)
#             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [128]:
best_step

array([23310, 25620, 27150, 28060, 31170, 31310, 32550, 35100, 35710,
       35740])